In [ ]:
import numpy as np
import mags
import obsSlots
import hp2np 
import hexalate


import decam2hp
import jsonMaker
from os import getenv
import pandas as pd
import sys
import  matplotlib.pyplot as plt

In [5]:
def get_hexinfo(ra, dec, prob, expTime, filt, mjd, best=False, camera="decam"):
    """
    Gets information about given hexes. Based on Jim's simplicity.calc
    code.

    Parameters:
    -----------
    ra: np.vector
        vector of hex ra
    dec: np.vector
        vector of hex dec
    prob: np.vector
        vector of hex summed prob from LIGO map
    expTime: np.vector
        vector of exposure times
    filt: str
        telescope filt.
    mjd: float
        modified julian date.
    best: bool
        True/False if use best or not
    camera: str
        telescope camera name to take in account.
        
    Outputs:
    --------
    hexinfo: pd.dataframe
        dataframe with each row representing a hex. contains ra, dec,
        prob, and rise and set times in MJD (minutes), and is sorted by 
        probability.
    """

    # find the highest prob hex
    best_ix = np.argmax(prob)

    # find night statistics
    night, sunset, sunrise = mags.findNightDuration(mjd, camera)
    night = np.float64(night)*24.
    sunset = np.float64(sunset)
    sunrise = np.float64(sunrise)

    # work every minute from sunset to sunrise
    mjd_list = np.arange(sunset, sunrise+1./1440., 1./1440.)
    
    # calculate the limiting magnitudes
    limit_mag = []
    best_limit_mag = []
    moon_sep = []
    moon_phase = []
    print(sunset)
    obs = mags.observed(ra,dec,prob, sunset, doMaps=False, verbose=False)
    for mjd in mjd_list :
        obs.resetTime(mjd)
        obs.limitMag(filt,exposure=expTime)
        limit_mag.append(obs.maglim)
        best_limit_mag.append(obs.maglim[best_ix])
        moon_sep.append(obs.moonSep[0]*360./2/np.pi)
        moon_phase.append(obs.moonPhase)
    limit_mag = np.vstack(limit_mag)
    ix = limit_mag < 0; limit_mag[ix] = 0
    
    #copied from Jim's code, left it here in case someone wants moon information
    moon_sep = np.array(moon_sep)
    moon_phase = np.array(moon_phase)
    # moon phase: where 0 = full, 90 equals half, and  180 = new
    # convert to %full
    moon_phase = ((180.-moon_phase)/180.)*100.
    bins = np.arange(21,25.5,0.5)
    
    set_list = [ ]
    rise_list = [ ]

    for i in range(len(limit_mag)):
        for j in range(len(limit_mag[i])):
            if limit_mag[i][j] !=0 and limit_mag[i-1][j] == 0:
                rise_list.append([mjd_list[i], ra[j], dec[j]])
            elif limit_mag[i-1][j] !=0 and limit_mag[i][j] == 0:
                set_list.append([mjd_list[i-1], ra[j], dec[j]])
                #print(f'setting j={j}, i={i}, {ra[j]}, {mjd_list[i]}')
    
    set_times = []
    rise_times = []
    #print(rise_list)
    
    
    for i in range(len(ra)):
        sets =[]
        rises=[]
        
        #find rise and set times corresponding to each ra and dec
        for s in set_list:
            #print(s[1])
            if s[1]==ra[i] and s[2]==dec[i]:
                #print(s[0], s[1], ra[i], dec[i])
                sets.append(s[0])
                #print(s[1], s[2], ra[i], dec[i], s[0])
        #find rise time for each ra and dec
        for r in rise_list:
            if r[1]==ra[i] and r[2]==dec[i]:
#                print(f'For {ra[i]}, have {r[0]}')
                rises.append(r[0])

        if len(sets) != 0:
            set_times.append(np.max(sets))
        elif len(sets) == 0:
            set_times.append('N/A')
        if len(rises) !=0:
            rise_times.append(np.min(rises))
        elif len(rises) == 0:
            rise_times.append('N/A')

    #print(rise_times, set_times)
    rise_mins = 1440*(rise_times - sunset)
    set_mins = 1440*(set_times - sunset)
    #create the dataframe
    d = {'RA (deg)': ra, 'DEC (deg)': dec, 'PROBABILITY': prob, 'RISES (MIN SINCE SUNSET)': rise_mins, 'SETS (MIN SINCE SUNSET)':set_mins, 'RISES (MJD)': rise_times, 'SETS (MJD)': set_times}
    info = pd.DataFrame.from_dict(d)
    
    #sort by probability 
    hexinfo = info.sort_values(by='PROBABILITY', ascending=False)
#    print(hexinfo)
    
    return hexinfo

In [6]:
dec = np.linspace(0, 30, num=10)
ra = np.linspace(300, 340, num=10)
probs=np.random.randint(10, size=(10))


In [8]:
get_hexinfo(ra, dec, probs, 90, 'r', 60069, best=False, camera="decam")

60068.95138888889
Loading dust map: .//data/plank-ebv-HFI_CompMap_ThermaDustModel.fits
loading inverse stellar density = probability of recognition map
	 ... the sun is up
	 ... the sun is up


,RA (deg),DEC (deg),PROBABILITY,RISES (MIN SINCE SUNSET),SETS (MIN SINCE SUNSET),RISES (MJD),SETS (MJD)
5,322.222222,16.666667,9,549.000002,700.000002,60069.332639,60069.4375
8,335.555556,26.666667,9,652.000002,700.000002,60069.404167,60069.4375
0,300.000000,0.000000,8,407.000001,700.000002,60069.234028,60069.4375
3,313.333333,10.000000,8,489.000002,700.000002,60069.290972,60069.4375
1,304.444444,3.333333,6,434.000001,700.000002,60069.252778,60069.4375
4,317.777778,13.333333,6,519.000002,700.000002,60069.311806,60069.4375
7,331.111111,23.333333,5,615.000002,700.000002,60069.378472,60069.4375
9,340.000000,30.000000,3,693.000002,700.000002,60069.432639,60069.4375
2,308.888889,6.666667,2,461.000002,700.000002,60069.271528,60069.4375
6,326.666667,20.000000,1,581.000002,700.000002,60069.354861,60069.4375
